Data Engineering - Skilling - Data Analysis and SQL - Replace case clause with pattern tables - DuckDB
======================================================================================================

# References
* [Data Engineering helpers - Skilling - Data Analysis with SQL - Redame](https://github.com/data-engineering-helpers/data-engineering-skilling/blob/main/Data-Analysis-SQL.md#replace-case-clause-with-pattern-tables)
  * [Data Engineering helpers - Skilling - Data Analysis with SQL - Original Jupyter notebook with DuckDB (this notebook)](https://github.com/data-engineering-helpers/data-engineering-skilling/blob/main/notebooks/data-analysis-sql-001-replace-case-with-pattern-table-duckdb.ipynb)
  * [Data Engineering helpers - Skilling - Data Analysis with SQL - Jupyter notebook adapted to PySpark](https://github.com/data-engineering-helpers/data-engineering-skilling/blob/main/notebooks/data-analysis-sql-001-replace-case-with-pattern-table-spark.ipynb)
* Date: Aug. 2025
* Author: Matt Martin
  ([Matt Martin on LinkedIn](https://www.linkedin.com/in/mattmartin14/),
   [Matt Martin's Substack blog](https://substack.com/@performancede))
* Article on Substack: https://performancede.substack.com/p/handling-fuzzy-matching-of-transactions
* [Original Git repository with the code (Jupyter notebook)](https://github.com/mattmartin14/dream_machine/blob/main/substack/articles/2025.08.15-fuzzy_like_matching/scratchpad.ipynb)


# Setup

In [1]:
import sys, platform
print(sys.version)
print(platform.python_version())

3.12.11 (main, Jun 16 2025, 19:21:30) [Clang 17.0.0 (clang-1700.0.13.5)]
3.12.11


In [14]:
import duckdb
cn = duckdb.connect()

# Table/temporary view creations

## Grocery categories: pattern matching table

In [15]:
sql = """
    create or replace table grocery_categories (
        item_pattern text,
        item_category text
    )
"""
cn.execute(sql)

sql = """
INSERT INTO grocery_categories (item_pattern, item_category) VALUES
    ('%banana%', 'fruit'),
    ('%apple%', 'fruit'),
    ('%orange%', 'fruit'),
    ('%grape%', 'fruit'),
    ('%strawberry%', 'fruit'),
    ('%lettuce%', 'vegetable'),
    ('%spinach%', 'vegetable'),
    ('%carrot%', 'vegetable'),
    ('%broccoli%', 'vegetable'),
    ('%cucumber%', 'vegetable'),
    ('%chicken%', 'meat'),
    ('%beef%', 'meat'),
    ('%pork%', 'meat'),
    ('%salmon%', 'seafood'),
    ('%shrimp%', 'seafood'),
    ('%milk%', 'dairy'),
    ('%cheese%', 'dairy'),
    ('%yogurt%', 'dairy'),
    ('%bread%', 'bakery'),
    ('%bagel%', 'bakery');
"""
cn.execute(sql)

In [18]:
cn.sql("select * from grocery_categories").show()

┌──────────────┬───────────────┐
│ item_pattern │ item_category │
│   varchar    │    varchar    │
├──────────────┼───────────────┤
│ %banana%     │ fruit         │
│ %apple%      │ fruit         │
│ %orange%     │ fruit         │
│ %grape%      │ fruit         │
│ %strawberry% │ fruit         │
│ %lettuce%    │ vegetable     │
│ %spinach%    │ vegetable     │
│ %carrot%     │ vegetable     │
│ %broccoli%   │ vegetable     │
│ %cucumber%   │ vegetable     │
│ %chicken%    │ meat          │
│ %beef%       │ meat          │
│ %pork%       │ meat          │
│ %salmon%     │ seafood       │
│ %shrimp%     │ seafood       │
│ %milk%       │ dairy         │
│ %cheese%     │ dairy         │
│ %yogurt%     │ dairy         │
│ %bread%      │ bakery        │
│ %bagel%      │ bakery        │
├──────────────┴───────────────┤
│ 20 rows            2 columns │
└──────────────────────────────┘



## Transaction/main table

In [16]:
# Create the txns table
cn.execute("""
    CREATE OR REPLACE TABLE txns (
        pstd_dt DATE,
        item_desc TEXT,
        charge_amt DECIMAL(10,2)
    )
""")

# Insert records with hardcoded random dates
cn.execute("""
    INSERT INTO txns (pstd_dt, item_desc, charge_amt) VALUES
        ('2024-05-01', '4 Pack Banana', 2.99),
        ('2024-05-03', 'Shrimp 3 lbs', 15.50),
        ('2024-05-05', 'organic Apple', 1.25),
        ('2024-05-07', 'Whole milk', 3.49),
        ('2024-05-09', 'Bagel Dozen', 5.99),
        ('2024-05-11', 'CHEDDAR cheese', 4.25),
        ('2024-05-13', 'Baby Spinach', 2.50),
        ('2024-05-15', 'Carrot bunch', 1.99),
        ('2024-05-17', 'Beef Steak', 12.99),
        ('2024-05-19', 'Salmon Fillet', 13.75),
        ('2024-05-21', 'Grape Bunch', 3.10),
        ('2024-05-23', 'Lettuce Head', 1.75),
        ('2024-05-25', 'Pork Chops', 9.50),
        ('2024-05-27', 'YOGURT cup', 1.10),
        ('2024-05-29', 'Cucumber', 1.00),
        ('2024-06-01', 'Chicken breast', 8.25),
        ('2024-06-03', 'Strawberry Box', 4.00),
        ('2024-06-05', 'Orange Juice', 3.99),
        ('2024-06-07', 'Whole Wheat bread', 2.50),
        ('2024-06-09', 'Broccoli Crown', 2.20),
        ('2024-06-11', 'Bagel SANDWICH', 6.50),
        ('2024-06-13', 'Spinach Salad', 5.00),
        ('2024-06-15', 'Cheese PLATTER', 10.00),
        ('2024-06-17', 'Milk Chocolate', 2.75),
        ('2024-06-19', 'Apple Pie', 7.00),
        ('2024-06-21', 'Shrimp COCKTAIL', 14.00),
        ('2024-06-23', 'Banana Bread', 4.50),
        ('2024-06-25', 'Beef BURGER', 11.00),
        ('2024-06-27', 'PORK ribs', 13.00),
        ('2024-06-29', 'Grape Jelly', 2.80),
""")

In [19]:
cn.sql("select * from txns").show()

┌────────────┬─────────────────┬───────────────┐
│  pstd_dt   │    item_desc    │  charge_amt   │
│    date    │     varchar     │ decimal(10,2) │
├────────────┼─────────────────┼───────────────┤
│ 2024-05-01 │ 4 Pack Banana   │          2.99 │
│ 2024-05-03 │ Shrimp 3 lbs    │         15.50 │
│ 2024-05-05 │ organic Apple   │          1.25 │
│ 2024-05-07 │ Whole milk      │          3.49 │
│ 2024-05-09 │ Bagel Dozen     │          5.99 │
│ 2024-05-11 │ CHEDDAR cheese  │          4.25 │
│ 2024-05-13 │ Baby Spinach    │          2.50 │
│ 2024-05-15 │ Carrot bunch    │          1.99 │
│ 2024-05-17 │ Beef Steak      │         12.99 │
│ 2024-05-19 │ Salmon Fillet   │         13.75 │
│     ·      │       ·         │            ·  │
│     ·      │       ·         │            ·  │
│     ·      │       ·         │            ·  │
│ 2024-06-11 │ Bagel SANDWICH  │          6.50 │
│ 2024-06-13 │ Spinach Salad   │          5.00 │
│ 2024-06-15 │ Cheese PLATTER  │         10.00 │
│ 2024-06-17 │ Milk 

# Category resolution with the join on pattern table (rather than case clause)

In [20]:
sql = """
    select month(t.pstd_dt) as pstd_month
        ,gc.item_category as grocery_category
        ,sum(t.charge_amt) as total_charges
    from txns as t
        left join grocery_categories as gc 
            on t.item_desc ilike gc.item_pattern
    group by all
    order by pstd_month asc, total_charges desc
"""
cn.sql(sql).show()

┌────────────┬──────────────────┬───────────────┐
│ pstd_month │ grocery_category │ total_charges │
│   int64    │     varchar      │ decimal(38,2) │
├────────────┼──────────────────┼───────────────┤
│          5 │ seafood          │         29.25 │
│          5 │ meat             │         22.49 │
│          5 │ dairy            │          8.84 │
│          5 │ fruit            │          7.34 │
│          5 │ vegetable        │          7.24 │
│          5 │ bakery           │          5.99 │
│          6 │ meat             │         32.25 │
│          6 │ fruit            │         22.29 │
│          6 │ seafood          │         14.00 │
│          6 │ bakery           │         13.50 │
│          6 │ dairy            │         12.75 │
│          6 │ vegetable        │          7.20 │
├────────────┴──────────────────┴───────────────┤
│ 12 rows                             3 columns │
└───────────────────────────────────────────────┘

